# Utils functions

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from google.colab import drive

from keras.utils import np_utils

from tensorflow.keras.backend import clear_session
from tensorflow import Tensor
from tensorflow.keras import regularizers, utils
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.initializers import he_normal, random_normal
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
!pip install -q -U keras-tuner
import keras_tuner as kt

In [ ]:
drive.mount('/content/drive')
raw_data_pd = pd.read_pickle(r'drive/MyDrive/AML_2022-2023/data_raw.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def draw_by_index(data_obj, idx, draw_legend=True):
  plt.plot(data_obj['x-axis'][idx], data_obj['spectra'][idx], label=f'{data_obj["user"][idx]} ({idx})')
  if draw_legend:
    plt.legend()

def draw_by_user(data_obj, user):
  for i, x in enumerate(data_obj['user']):
    if x == user:
      plt.plot(data_obj['x-axis'][i], data_obj['spectra'][i])

def draw_by_category(data_obj, category):
  for i, x in enumerate(data_obj['category']):
    if x == category:
      plt.plot(data_obj['x-axis'][i], data_obj['spectra'][i])

def is_outlier(data_obj, idx, treshold=1200, val_out=64500):
  spectra = data_obj['spectra'][idx]
  x = data_obj['x-axis'][idx]
  ma_arr = np.ma.masked_less(x, treshold)
  masked = np.ma.masked_where(np.ma.getmask(ma_arr), spectra)
  return masked.max() == val_out

def how_many_outliers(data_obj, treshold=1200, val_out=64500):
  count = 0
  for idx, _ in enumerate(data_obj['user']):
    if is_outlier(data_obj, idx, treshold, val_out):
      count += 1
  return count

def get_dataset_no_outliers(data_obj, treshold=1200, val_out=64500):
  new_df = data_obj.copy()
  idexes_outliers = []
  for idx, _ in enumerate(data_obj['user']):
    if is_outlier(data_obj, idx, treshold, val_out):
      idexes_outliers.append(idx)
  return new_df.drop(idexes_outliers).reset_index()

def get_dataset_fitted(data_obj, deg=4, val_out=64500):
  new_df = data_obj.copy()
  for idx, _ in enumerate(new_df['user']):
    y = get_fitted(new_df, idx, deg, val_out)
    #new_df.loc[idx, 'spectra'] = new_df['spectra'][idx] - y
    new_df['spectra'][idx] = new_df['spectra'][idx] - y
  return new_df

def get_fitted(data_obj, idx, deg=5, val_out=64500):
  y = data_obj['spectra'][idx]
  my = np.ma.masked_equal(y, val_out)
  x = data_obj['x-axis'][idx]
  mx = np.ma.masked_where(np.ma.getmask(my), x)
  poly = np.polyfit(mx, my, deg=deg)
  return np.polyval(poly, mx).filled(val_out)

def draw_by_index_fitted(data_obj, idx, deg=5, draw_legend=True):
  x = data_obj['x-axis'][idx]
  y = data_obj['spectra'][idx]
  fit = get_fitted(data_obj, idx, deg)
  plt.plot(x, y-fit, label=f'{data_obj["user"][idx]} ({idx})')
  if draw_legend:
    plt.legend()

def draw_by_user_fitted(data_obj, user, deg=5):
  for i, x in enumerate(data_obj['user']):
    if x == user:
      draw_by_index_fitted(data_obj, i, deg, False)

def draw_by_category_fitted(data_obj, category, deg=5):
  for i, x in enumerate(data_obj['category']):
    if x == category:
      draw_by_index_fitted(data_obj, i, deg, False)

def plot_history(history, validation=True):
  
  fig, axs = plt.subplots(1, 2, figsize=(20,5))

  axs[0].plot(history.history['loss'])
  if validation:
    axs[0].plot(history.history['val_loss'])
    #overfitepoch = np.where(np.array(history.history['val_loss']) == min(history.history['val_loss']))
    #axs[0].vlines(overfitepoch, 0, 1.5) 
  axs[0].set_title('Loss')
  axs[0].set_ylabel('Loss')
  axs[0].set_xlabel('Epoch')
  axs[0].grid()

  axs[1].plot(history.history['categorical_accuracy'])
  if validation:
    axs[1].plot(history.history['val_categorical_accuracy'])
    #axs[1].vlines(overfitepoch, 0.6, 1)  
  axs[1].set_title('Categorical accuracy')
  axs[1].set_ylabel('Categorical accuracy')
  axs[1].set_xlabel('Epoch')
  axs[1].grid()

  if validation:
    plt.figlegend(['Train set', 'Validation set'], loc='upper center',
                  fontsize=14, ncol=2)

  plt.show()

  # Print final values
  print('At the',len(history.history['loss']), 'th epoch: \n')
  print('The loss:', history.history['loss'][-1])
  print('The categorical_accuracy', history.history['categorical_accuracy'][-1])
  if validation:
    print('The val_loss:', history.history['val_loss'][-1])
    print('The val_categorical_accuracy:', history.history['val_categorical_accuracy'][-1])


def plot_history_bin(history, validation=True):
  
  fig, axs = plt.subplots(1, 2, figsize=(20,5))

  axs[0].plot(history.history['loss'])
  if validation:
    axs[0].plot(history.history['val_loss'])
    #overfitepoch = np.where(np.array(history.history['val_loss']) == min(history.history['val_loss']))
    #axs[0].vlines(overfitepoch, 0, 1.5) 
  axs[0].set_title('Loss')
  axs[0].set_ylabel('Loss')
  axs[0].set_xlabel('Epoch')
  axs[0].grid()

  axs[1].plot(history.history['accuracy'])
  if validation:
    axs[1].plot(history.history['val_accuracy'])
    #axs[1].vlines(overfitepoch, 0.6, 1)  
  axs[1].set_title('Accuracy')
  axs[1].set_ylabel('Accuracy')
  axs[1].set_xlabel('Epoch')
  axs[1].grid()

  if validation:
    plt.figlegend(['Train set', 'Validation set'], loc='upper center',
                  fontsize=14, ncol=2)

  plt.show()

  # Print final values
  print('At the',len(history.history['loss']), 'th epoch: \n')
  print('The loss:', history.history['loss'][-1])
  print('The accuracy', history.history['accuracy'][-1])
  if validation:
    print('The val_loss:', history.history['val_loss'][-1])
    print('The val_accuracy:', history.history['val_accuracy'][-1])

In [ ]:
def get_preprocessed_data_1d(raw_dataset, train_pr=0.5, val_pr=0.25, test_pr=0.25, augment=False):
  clean_dataset = get_dataset_no_outliers(raw_dataset, treshold=400)

  x_train, y_train, x_val, y_val, x_test, y_test = dataset_split(clean_dataset, 
                                                                 train_pr=train_pr,
                                                                 val_pr=val_pr,
                                                                 test_pr=test_pr)
  x_train = normalize(x_train)
  x_val = normalize(x_val)
  x_test_norm = normalize(x_test)

  divided_spectra = divide_spectra(x_test, x_test_norm, raw_dataset)

  if augment:
    x_train, y_train = data_augmentation(x_train, y_train)
  
  return x_train, y_train, x_val, y_val, x_test, y_test, x_test_norm, divided_spectra

In [ ]:
def get_user(spectra, data, tupled):
  idx = raw_data_pd.index[tupled == tuple(spectra)][0]
  return data['user'][idx]

def divide_spectra(x_test, x_test_norm, data):
  results = {}
  patients = data['user'].unique()
  tupled = raw_data_pd.spectra.map(tuple)
  for p in patients:
    results[p] = []

  for idx, spectra in enumerate(x_test):
    user = get_user(spectra, data, tupled)
    results[user].append(x_test_norm[idx])

  return {k:v for k,v in results.items() if v}

In [ ]:
def get_label(patients):
  
  label=[]
  for p in patients:
    if 'COV+' in p:
      label.append(0)
    
    elif 'COV-' in p:
      label.append(1)
    
    elif 'CTRL' in p:
      label.append(2)

  return label

In [ ]:
def dataset_split(data, train_pr, val_pr, test_pr):

  patients = data['user'].unique()
  val_pr=val_pr*(1+test_pr)
  # x1: train, x2: val, x3: test
  x1, x3 = train_test_split(patients, test_size=test_pr, stratify=get_label(patients))
  x1, x2= train_test_split(x1, test_size=val_pr, stratify=get_label(x1))

  x_train=[]
  x_val=[]
  x_test=[]

  train_users = []
  test_users = []
  val_users = []

  for idx, user in enumerate(raw_data_pd['user']):
    if user in x1:
      x_train.append(raw_data_pd['spectra'][idx])
      train_users.append(raw_data_pd['user'][idx])
    elif user in x2:
      x_val.append(raw_data_pd['spectra'][idx])
      val_users.append(raw_data_pd['user'][idx])
    elif user in x3:
      x_test.append(raw_data_pd['spectra'][idx])
      test_users.append(raw_data_pd['user'][idx])

  y_train=get_label(train_users)
  y_val=get_label(val_users)
  y_test=get_label(test_users)

  y_train = utils.to_categorical(y_train, 3)
  y_test = utils.to_categorical(y_test, 3)
  y_val = utils.to_categorical(y_val, 3)

  x_train = np.array(x_train)
  x_val = np.array(x_val)
  x_test = np.array(x_test)

  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
def normalize(x):

  new_x=[]

  for spectra in x:
    #new_x.append(spectra/max(spectra))
    new_x.append(spectra/spectra[0])
    #new_x.append(spectra/np.mean(spectra))
    
  return np.array(new_x)
#gaussian noise

def data_augmentation(x, y, n_spectra=5):
  new_x = []
  new_y = []

  for idx, s in enumerate(x):
    for n in range(n_spectra):
      noise = np.random.normal(0,0.0005,991)
      new_spectra = s+noise
      new_x.append(new_spectra)
      new_y.append(y[idx])

  return np.array(new_x), np.array(new_y)

In [ ]:
def patients_results(model, spectra_patients, majority=False):
  results = {
    "user": [],
    "prob_cov+": [],
    "prob_cov-": [],
    "prob_ctrl": [],
    "prediction": [],
    "correct_value": []
  }
  for patient in spectra_patients.keys():
    r = model.predict(np.array(spectra_patients[patient]), verbose=0)
    
    if majority:
      r = np.argmax(r, axis=1)
      count=[]

      for i in range(3):
        count.append((r==i).sum())

      pred = np.argmax(count)
    
    else:
      r = r.sum(axis=0)
      r = r / len(spectra_patients[patient])
      pred = r.argmax()
        
    
    if pred == 0:
      pred = 'COV+'
    elif pred == 1:
      pred = 'COV-'
    elif pred == 2:
      pred = 'CTRL'

    if 'COV+' in patient:
      cor = 'COV+'
    elif 'COV-' in patient:
      cor = 'COV-'
    elif 'CTRL' in patient:
      cor = 'CTRL'

    results["user"].append(patient)
    results["prob_cov+"].append(r[0])
    results["prob_cov-"].append(r[1])
    results["prob_ctrl"].append(r[2])
    results["prediction"].append(pred)
    results["correct_value"].append(cor)

  return pd.DataFrame(results)

In [ ]:
def patients_accuracy_tern(results):

  correct = 0

  for idx, _ in enumerate(results['user']):
    if results['prediction'][idx] == results['correct_value'][idx]:
      correct += 1

  return correct/len(results)

def patients_accuracy(results):
  
  correct = 0

  for idx, _ in enumerate(results['user']):

    if results['prediction'][idx]=='COV+':
      if results['correct_value'][idx]=='COV+':
        correct += 1
    else:
      if results['correct_value'][idx] != 'COV+':
        correct+=1

  return correct/len(results)

def patients_sensitivity(results):

  true_positive = 0
  false_negative = 0

  for idx, _ in enumerate(results['user']):
    pred = results['prediction'][idx]
    res = results['correct_value'][idx]
    if pred == 'COV+' and res == 'COV+':
      true_positive += 1
    elif pred != 'COV+' and res == 'COV+':
      false_negative += 1

  try:
    return true_positive/(true_positive+false_negative)
  except:
    return float('NaN')

def patients_specifity(results):

  true_negative = 0
  false_positive = 0

  for idx, _ in enumerate(results['user']):
    pred = results['prediction'][idx]
    res = results['correct_value'][idx]
    if pred != 'COV+' and res != 'COV+':
      true_negative += 1
    elif pred == 'COV+' and res != 'COV+':
      false_positive += 1

  try:
    return true_negative/(true_negative+false_positive)
  except:
    return float('NaN')

In [ ]:
def patients_confusion_matrix(results):

  y_pred = results['prediction']
  y_true = results['correct_value']

  label=['COV+', 'COV-', 'CTRL']

  mat=ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred), display_labels=label)
  mat.plot()
  plt.show()

In [ ]:
def divide_patients_in_category(patients):
  cov_p = []
  cov_m = []
  ctrl = []

  for p in patients:
    if 'COV+' in p:
      cov_p.append(p)
    elif 'COV-' in p:
      cov_m.append(p)
    elif 'CTRL' in p:
      ctrl.append(p)
  return cov_p, cov_m, ctrl

In [ ]:
def divide_patients_in_groups(patients, k=1):
  
  cov_p, cov_m, ctrl = divide_patients_in_category(patients)


  random.shuffle(cov_p)
  random.shuffle(cov_m)
  random.shuffle(ctrl)
  
  total_num = len(cov_p) 

  new_k = int(np.floor(k / 3))

  new_division = []

  for i in np.arange(0,total_num, new_k, dtype='int32'):
    temp_y = []
    temp_y.append(cov_p[i:i+new_k])
    temp_y.append(cov_m[i:i+new_k])
    temp_y.append(ctrl[i:i+new_k])

    new_division.append(np.reshape(temp_y, (len(temp_y)*new_k, ) ))
  return new_division

In [ ]:
def get_split_by_names(odataset, test_names):

  dataset = odataset.copy()
  x_test = []
  y_test = []

  x_train = []
  y_train = []

  for idx, name in enumerate(dataset['user']):
    if name in test_names:
      x_test.append(dataset['spectra'][idx])
      y_test.append(name)
    else:
      x_train.append(dataset['spectra'][idx])
      y_train.append(name)

  y_train=get_label(y_train)
  y_test=get_label(y_test)

  return x_train, y_train, x_test, y_test

In [ ]:
def train_lopocv(raw_dataset, make_model, augment=False, k=1):
  dataset = get_dataset_no_outliers(raw_dataset, treshold=400)

  patients = dataset.user.unique()
  division = divide_patients_in_groups(patients, k)

  accuracies_tern = []
  accuracies = []
  sensitivity = []
  specificity = []

  maj_accuracies_tern = []
  maj_accuracies = []
  maj_sensitivity = []
  maj_specificity = []

  for idx, testset in enumerate(division):
    x_train, y_train, x_test, y_test = get_split_by_names(dataset, testset)

    x_train = normalize(np.array(x_train))
    x_test_norm = normalize(np.array(x_test))

    divided_spectra = divide_spectra(x_test, x_test_norm, raw_dataset)

    if augment:
      x_train, y_train = data_augmentation(x_train, y_train)
  
    y_train = utils.to_categorical(y_train, 3)
    y_test = utils.to_categorical(y_test, 3)

    x_train = np.array(x_train)
    x_test = np.array(x_test)


    batch_size = 32
    n_epochs = 15
    
    checkpoint_path = '/tmp/checkpoint' 
    model_checkpoint_callback = ModelCheckpoint(
      filepath=checkpoint_path,
      save_weights_only=True,
      monitor='categorical_accuracy',
      mode='max',
      save_best_only=True)
  
    model_earlystopping_callback = EarlyStopping(
      monitor='categorical_accuracy',
      min_delta=0.01,
      patience=50,
      verbose=1
    )

    clear_session()
    model = make_model()
    network_history = model.fit(x_train, y_train, batch_size=batch_size, 
                                callbacks=[model_checkpoint_callback,
                                           model_earlystopping_callback],
                                epochs=n_epochs, verbose=0,
                                validation_data=(x_test_norm, y_test))
    model.load_weights(checkpoint_path)
    #plot_history(network_history, False)
  
    df_res = patients_results(model, divided_spectra, majority=False)
    accuracies_tern.append(patients_accuracy_tern(df_res))
    accuracies.append(patients_accuracy(df_res))
    sensitivity.append(patients_sensitivity(df_res))
    specificity.append(patients_specifity(df_res))

    maj_df_res = patients_results(model, divided_spectra, majority=True)
    maj_accuracies_tern.append(patients_accuracy_tern(maj_df_res))
    maj_accuracies.append(patients_accuracy(maj_df_res))
    maj_sensitivity.append(patients_sensitivity(maj_df_res))
    maj_specificity.append(patients_specifity(maj_df_res))



    print(f'\nIterazione {idx}')
    print(f'\tTrainAcc={network_history.history["categorical_accuracy"][-1]}')

    print(f'\nMajority=False')
    print(f'\tTernAcc={accuracies_tern[-1]}')
    print(f'\tBinAcc={accuracies[-1]}')
    print(f'\tSens={sensitivity[-1]}')
    print(f'\tSpec={specificity[-1]}')

    print(f'\nMajority=True')
    print(f'\tTernAcc={maj_accuracies_tern[-1]}')
    print(f'\tBinAcc={maj_accuracies[-1]}')
    print(f'\tSens={maj_sensitivity[-1]}')
    print(f'\tSpec={maj_specificity[-1]}')
    #print(df_res)

  return accuracies_tern, accuracies, sensitivity, specificity, maj_accuracies_tern, maj_accuracies, maj_sensitivity, maj_specificity

# Hyperband

In [ ]:
power2=[2**i for i in range(3,10)]
power2

[8, 16, 32, 64, 128, 256, 512]

In [ ]:
def create_VGG(hp):
    
  f1=hp.Choice('f1', power2)
  f2=hp.Choice('f2', power2)
  f3=hp.Choice('f3', power2)
  gaus=hp.Float('gaus', min_value=0., max_value=0.1, step=0.005)
  dense=hp.Int('dense', min_value=20, max_value=120, step=10)

  inputs = Input(shape=(991, 1))
  t = GaussianNoise(gaus)(inputs)

  #block 1
  t = Conv1D(f1, kernel_size=3, strides=1, padding="same")(inputs)
  t = Conv1D(f1, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)
  
  #block 2
  t = Conv1D(f2, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(f2, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)
  
  #block 3
  t = Conv1D(f3, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(f3, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(f3, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)
  '''
  #block 4
  t = Conv1D(512, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(512, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(512, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)

  #block 5 
  t = Conv1D(512, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(512, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(512, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)
  '''

  t = Flatten()(t)
  t = Dense(dense, activation='relu')(t)
  t = Dense(dense, activation='relu')(t)
  outputs = Dense(3, activation='softmax')(t)
  
  model = Model(inputs, outputs)

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy']
  )

  return model

In [ ]:
tuner = kt.Hyperband(create_VGG,
                     objective='val_categorical_accuracy',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='prova1')

x_train, y_train, x_val, y_val, x_test, y_test, x_test_norm, divided_spectra = get_preprocessed_data_1d(raw_data_pd,
                                                                                                        augment=False)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val))
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)


In [ ]:
print(best_hps.get('f1'))
print(best_hps.get('f2'))
print(best_hps.get('f3'))
print(best_hps.get('gaus'))
print(best_hps.get('dense'))

# Training

In [ ]:
def make_model():

  f1 = 32 
  f2 = 8
  f3 = 8
  gaus = 0.06
  dense = 40

  inputs = Input(shape=(991, 1))

  t = GaussianNoise(gaus)(inputs)

  #block 1
  t = Conv1D(f1, kernel_size=3, strides=1, padding="same")(inputs)
  t = Conv1D(f1, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)
  
  #block 2
  t = Conv1D(f2, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(f2, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)
  
  #block 3
  t = Conv1D(f3, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(f3, kernel_size=3, strides=1, padding="same")(t)
  t = Conv1D(f3, kernel_size=3, strides=1, padding="same")(t)
  t = MaxPool1D(pool_size=2, strides=2)(t)


  t = Flatten()(t)
  t = Dense(dense, activation='relu')(t)
  t = Dense(dense, activation='relu')(t)
  outputs = Dense(3, activation='softmax')(t)
  
  model = Model(inputs, outputs)

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy']
  )
  model.summary()
  
  return model

In [ ]:
model=make_model()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 991, 1)]          0         
                                                                 
 conv1d_7 (Conv1D)           (None, 991, 32)           128       
                                                                 
 conv1d_8 (Conv1D)           (None, 991, 32)           3104      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 495, 32)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 495, 8)            776       
                                                                 
 conv1d_10 (Conv1D)          (None, 495, 8)            200       
                                                           

In [ ]:
acc_tern, acc, sens, spec, m_acc_tern, m_acc, m_sens, m_spec = train_lopocv(raw_data_pd, make_model,
                                                                            k=3)
print('\n\n')
print(acc_tern)
print(acc)
print(sens)
print(spec)
print(f'\nAcc_tern: {np.mean(acc_tern)}')
print(f'Acc: {np.mean(acc)}')
print(f'Sens: {np.mean(sens)}')
print(f'Spec: {np.mean(spec)}')

print('\n\n')
print(m_acc_tern)
print(m_acc)
print(m_sens)
print(m_spec)
print(f'\nAcc_tern: {np.mean(m_acc_tern)}')
print(f'Acc: {np.mean(m_acc)}')
print(f'Sens: {np.mean(m_sens)}')
print(f'Spec: {np.mean(m_spec)}')


Iterazione 0
	TrainAcc=0.7956989407539368

Majority=False
	TernAcc=1.0
	BinAcc=1.0
	Sens=1.0
	Spec=1.0

Majority=Turne
	TernAcc=1.0
	BinAcc=1.0
	Sens=1.0
	Spec=1.0

Iterazione 1
	TrainAcc=0.7956896424293518

Majority=False
	TernAcc=0.6666666666666666
	BinAcc=0.6666666666666666
	Sens=0.0
	Spec=1.0

Majority=Turne
	TernAcc=0.3333333333333333
	BinAcc=0.3333333333333333
	Sens=0.0
	Spec=0.5

Iterazione 2
	TrainAcc=0.9060344696044922

Majority=False
	TernAcc=0.6666666666666666
	BinAcc=0.6666666666666666
	Sens=1.0
	Spec=0.5

Majority=Turne
	TernAcc=0.6666666666666666
	BinAcc=0.6666666666666666
	Sens=1.0
	Spec=0.5

Iterazione 3
	TrainAcc=0.7509677410125732

Majority=False
	TernAcc=1.0
	BinAcc=1.0
	Sens=1.0
	Spec=1.0

Majority=Turne
	TernAcc=1.0
	BinAcc=1.0
	Sens=1.0
	Spec=1.0

Iterazione 4
	TrainAcc=0.8438982367515564

Majority=False
	TernAcc=1.0
	BinAcc=1.0
	Sens=1.0
	Spec=1.0

Majority=Turne
	TernAcc=1.0
	BinAcc=1.0
	Sens=1.0
	Spec=1.0

Iterazione 5
	TrainAcc=0.7934454679489136

Majority=Fa